# 3. Užduotis

Modeliuokite a. d. $X$, turinčio binominį skirstinį su parametrais $k$ ir $p$, t. y.

$$
P(X = m | k, p) = C^m_k  p^m (1 − p)^{k−m},
$$

dydžio $n = 50$ paprastąją atsitiktinę imtį. Naudodami `R` programos funkciją `maxLik` arba
`optim` DT metodu įvertinkite parametrą $p$. Palyginkite su tikra parametro reikšme.

In [1]:
set.seed(314)

In [2]:
n <- 50
(p <- 1 / pi)
k <- 100
observ <- rbinom(n, k, p)

[1] 0.3183099

In [3]:
# Momentų metodas
p_hat_moment <- function(x) { 1 - var(x) / mean(x)}
# k_hat_moment <- function(x) { 
#    m <- mean(x)
#    m^2 / (m - var(x))
# }

In [9]:
binom_loglik_p <- function(observ) {
    n <- length(observ)
    m <- mean(observ)
    
    # Precalculate:
    tmp1 <- n * m
    tmp2 <- n * (k - m)
    tmp3 <- sum(lfactorial(k) - lfactorial(observ) - lfactorial(k - observ))

    function(theta) {
        p <- theta[1]
        tmp1 * log(p) + tmp2 * log(1 - p) + tmp3
    }
}

In [10]:
maxlik_optimiser <- function(f, start) {
    suppressPackageStartupMessages(require(maxLik))
    mle <- maxLik::maxLik(logLik = f, start = start)
    r <- list(
        estimates = coef(mle),
        hessian = hessian(mle)
    )
    return(r)
}

mle_estimator <- function(loglikelihood, optimiser) {
    function(data, start) {
        optimiser(loglikelihood(data), start)
    }
}

binom_mle_estimator <- mle_estimator(binom_loglik_p, maxlik_optimiser)

In [12]:
(start_p <- p_hat_moment(observ))
(theta_hat_mle <- binom_mle_estimator(observ, c(start_p)))
print(p)

[1] 0.4833167

-22836.5


[1] 0.3183099
